In [4]:
import scipy
from netCDF4 import Dataset
import numpy as np
from scipy.spatial import KDTree
from ll_xy import lonlat_to_xy
import itertools
import matplotlib.pyplot as plt
import datetime
import calendar
import pandas as pd
import h5py
import os
import pickle
import tqdm

server_dir = '/home/robbie/uit_mnt'

satam_directory = f'{server_dir}/Data/romal7177/ResearchData/IFT/EarthObservation/SatelliteAltimetry/'
scratch=f'{server_dir}/scratch/robbie/nrcs/pickles'

nh_it_dir=f'{satam_directory}OSISAF Sea Ice Type'
f = '2019/12/ice_type_nh_polstere-100_multi_201912201200.nc'
x = Dataset(f'{nh_it_dir}/{f}')
longrid = np.array(x['lon'])
latgrid = np.array(x['lat'])
nh_xgrid,nh_ygrid=lonlat_to_xy(longrid,latgrid,hemisphere='n')
nh_tree = KDTree(list(zip(nh_xgrid.ravel(),
                       nh_ygrid.ravel())))

sh_it_dir = f'{server_dir}/scratch/robbie/melsheimer_ice_type'
latf = 'south_lat_12km.hdf'
sh_lat = Dataset(f'{sh_it_dir}/{latf}')
v = list(sh_lat.variables)[0]
lat = np.array(sh_lat[v])

lonf = 'south_lon_12km.hdf'
sh_lon = Dataset(f'{sh_it_dir}/{lonf}')
v = list(sh_lon.variables)[0]
lon = np.array(sh_lon[v])

sh_xgrid,sh_ygrid=lonlat_to_xy(lon,lat,hemisphere='s')
sh_tree = KDTree(list(zip(sh_xgrid.ravel(),
                       sh_ygrid.ravel())))

trees = {'s':sh_tree,'n':nh_tree}
grids = {'s':sh_xgrid,'n':nh_xgrid}

cs_directory=f'{server_dir}/scratch/robbie/nrcs/h5_dir'
cs_files = os.listdir(cs_directory)
cs_files = [a for a in cs_files if a[:2]=='cs']

for file in cs_files[:]:
    
    print(file)
    
    hem = file.split('_')[1][0]
    datestr = file.split('_')[-1]
    year = datestr[:4]
    month = datestr[4:6]
    
    with h5py.File(f'{cs_directory}/{file}','r') as x:
        keys = list(x.keys())
    
    days = np.array([int(key[8:10]) for key in keys])
    
    num_days = np.max(days)
    
    mean_grids=[]
    
    for day in tqdm.trange(1,num_days+1):
        
        mean_vals = np.full(grids[hem].shape,np.nan)
        
        r=np.argwhere(days==day)
        
        
        if r.shape[0]:#Sometimes they just switch it off for a day...

            keys_on_date = keys[r[0][0]:r[-1][0]]

            basic_dict = {(a,b):[] for a,b in itertools.product(np.arange(grids[hem].shape[0]),
                                                                np.arange(grids[hem].shape[1]))}

            for key in keys_on_date:

                df = pd.read_hdf(f'{cs_directory}/{file}',mode='r',key=key)

                df = df[df['floe_height']>-100]

                df = df[df['pp']<9]

                at_x, at_y = lonlat_to_xy(np.array(df['lon']),
                                          np.array(df['lat']),
                                          hemisphere=hem)

                tree = trees[hem]

                dist,ind = tree.query(np.array([at_x,at_y]).T)

                ind2d = np.unravel_index(ind, grids[hem].shape)

                sig0_lin = 10 ** (df['sigma0']/10)

                for (a,b,s) in zip(ind2d[0],ind2d[1],sig0_lin):
                    basic_dict[(a,b)].append(s)


            vals,xscat,yscat = [],[],[]

            
            for key, value in basic_dict.items():


                if value:
                    mean_vals[key[0],key[1]] = np.nanmean(value)

        mean_grids.append(mean_vals)

    pickle.dump(mean_grids,open(f'{scratch}/cs2_{month}_{year}_{hem}h.p','wb'))
    pickle.dump(np.arange(1,num_days+1),open(f'{scratch}/cs2_{month}_{year}_{hem}h_dates.p','wb'))

cs2_sh_201712.h5


100%|███████████████████████████████████████████| 31/31 [02:34<00:00,  4.99s/it]


cs2_sh_201606.h5


100%|███████████████████████████████████████████| 30/30 [03:07<00:00,  6.26s/it]


cs2_sh_201807.h5


...

100%|███████████████████████████████████████████| 31/31 [03:22<00:00,  6.53s/it]


cs2_nh_202009.h5


100%|███████████████████████████████████████████| 30/30 [03:16<00:00,  6.53s/it]


cs2_nh_201101.h5


 97%|█████████████████████████████████████████▌ | 30/31 [03:47<00:07,  7.57s/it]

KeyboardInterrupt



# Now make code to read these pickles

In [15]:
keys

['nh20110101T000131',
 'nh20110101T000333',
 'nh20110101T004630',
 'nh20110101T005909',
 'nh20110101T010056',
 'nh20110101T013834',
 'nh20110101T014646',
 'nh20110101T021539',
 'nh20110101T022523',
 'nh20110101T023800',
 'nh20110101T031400',
 'nh20110101T031747',
 'nh20110101T032433',
 'nh20110101T032945',
 'nh20110101T033112',
 'nh20110101T033622',
 'nh20110101T040339',
 'nh20110101T041639',
 'nh20110101T045411',
 'nh20110101T045645',
 'nh20110101T050332',
 'nh20110101T050631',
 'nh20110101T050759',
 'nh20110101T051657',
 'nh20110101T054300',
 'nh20110101T055754',
 'nh20110101T063247',
 'nh20110101T063348',
 'nh20110101T063609',
 'nh20110101T063755',
 'nh20110101T064227',
 'nh20110101T064739',
 'nh20110101T072031',
 'nh20110101T073818',
 'nh20110101T075413',
 'nh20110101T081555',
 'nh20110101T082102',
 'nh20110101T083123',
 'nh20110101T085634',
 'nh20110101T090102',
 'nh20110101T091734',
 'nh20110101T093240',
 'nh20110101T093540',
 'nh20110101T095547',
 'nh20110101T095719',
 'nh201101